<a href="https://colab.research.google.com/github/ManjuRama/FinMath/blob/main/govLaw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Sample ISDA agreement data
data = {
    "file_name": ["file1", "file2", "file3", "file4", "file5"],
    "category": ["ISDA", "Amendment", "ISDA", "Amendment", "Amendment"],
    "date": ["2023-05-01", "2023-06-15", "2023-04-01", "2023-07-20", "2023-08-01"],
    "date_of_original_agreement": [None, "2023-05-01", None, "2023-04-01", "2023-04-01"],
    "summary": [
        "Initial agreement signed.",
        "Governing Law changed to NY law.",
        "Another agreement signed.",
        "Governing Law changed to UK law.",
        "Governing Law changed to EU law."
    ],
    "gov_law": ["US law", "NY law", "UK law", "UK law", "EU law"]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Convert dates to datetime for easier processing
df['date'] = pd.to_datetime(df['date'])
df['date_of_original_agreement'] = pd.to_datetime(df['date_of_original_agreement'], errors='coerce')

# Function to find the most recent amendment for each original agreement
def update_gov_law(df):
    # Group amendments by their original agreement date
    amendments = df[df['category'] == "Amendment"]

    # Keep the most recent amendment for each original agreement
    most_recent_amendments = (
        amendments
        .sort_values(by="date", ascending=False)  # Sort by date descending
        .groupby("date_of_original_agreement")
        .first()  # Take the most recent amendment for each original agreement
    )

    # Apply changes to the original agreements
    for original_date, amendment_row in most_recent_amendments.iterrows():
        if pd.notna(original_date):
            # Find the original agreement
            original_agreement = df[
                (df['date'] == original_date) &
                (df['category'] == "ISDA")
            ]
            if not original_agreement.empty:
                original_index = original_agreement.index[0]
                # Update the original agreement's governing law using 'gov_law' of the most recent amendment
                df.at[original_index, 'final_gov_law'] = amendment_row['gov_law']

# Initialize the new column with existing governing law
df['final_gov_law'] = df['gov_law']

# Apply the update function
update_gov_law(df)

# Fill NaN in final_gov_law (if any) with placeholders
df['final_gov_law'] = df['final_gov_law'].fillna("No Change")

# Display the updated DataFrame
print(df)
